In [ ]:
import os
import pandas as pd
import numpy as np
from mol_lib import xtb_isomers, FromXtbToSpinM, FromFormulaToAtomsList, FromFormulaToElectronCore, FromFormulaToElectron
import matplotlib.pyplot as plt

with open("/Users/tinaccil/Documents/GitHub/woon_query/data_analysis/dataframe/molecules_info.csv","r") as data:
	df_mol = pd.read_csv(data, delimiter = '\t')
with open("/Users/tinaccil/Documents/GitHub/woon_query/data_analysis/dataframe/network.csv","r") as data:
	df_net = pd.read_csv(data, delimiter = '\t')

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_mol)

In [ ]:
FromFormulaToElectronCore
df_mol['core_el'] = df_mol.apply(lambda row: FromFormulaToElectronCore(row['formula']), axis=1)
df_mol['electron'] = df_mol.apply(lambda row: FromFormulaToElectron(row['formula']), axis=1)
df_mol.sort_values(by=['electron',"Mass"], inplace=True)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(df_mol)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(X, a, b, c):
    x,y  = X
    return a * x + b * y + c

name = df_mol['name'].tolist()
e_xtb = df_mol['energy(kJ/mol)_xtb'].tolist()
e_w = df_mol['energy(kJ/mol)_woon'].tolist()
e_ce = df_mol['core_el'].tolist()
e_e = df_mol['electron'].tolist()


params, params_covariance = curve_fit(func, (e_xtb,e_e), e_w)
print(params)

e_xtb_n = []
for i,item in enumerate(e_xtb):
    e_xtb_n.append(func((e_xtb[i],e_e[i]), params[0], params[1], params[2]))

plt.figure(figsize=(20, 10))
plt.scatter(name, e_w, label='woon', color='Red')
plt.scatter(name, e_xtb_n, label='f(xtb,ncore)', color='Blue')
plt.legend(loc='best')
plt.xticks(rotation=90)
#plt.ylim(1.45e-10,2.01e-10)
plt.show()   

In [ ]:
ax = df_mol.plot(x='name', y='energy(kJ/mol)_woon', kind='scatter', color='Blue', label='Woon')
df_mol.plot(x='name', y='energy(kJ/mol)_xtb', color='Red', kind='scatter',ax=ax, label='xTB', figsize=(20,10))
plt.xticks(rotation=90)

In [ ]:
#df_mol.plot(x='energy(kJ/mol)_woon', y='energy(kJ/mol)_xtb', color='Red', kind='scatter', figsize=(20,10))

fig, ax = plt.subplots(figsize=(30, 60))
ax.scatter(e_w, e_xtb)
for i, txt in enumerate(name):
    ax.annotate(txt, (e_w[i], e_xtb[i]))

In [ ]:
def norm(val,val_n):
    tmp = (val - val_n) / abs(val_n) 
    return(tmp)

max_val_x = df_mol['energy(kJ/mol)_xtb'].min()
df_mol['n_x'] = df_mol.apply(lambda row: norm(row['energy(kJ/mol)_xtb'],max_val_x), axis=1)

max_val_w = df_mol['energy(kJ/mol)_woon'].min()
df_mol['n_w'] = df_mol.apply(lambda row: norm(row['energy(kJ/mol)_woon'],max_val_w), axis=1)

ax = df_mol.plot(x='name', y='n_w', kind='scatter', color='Blue', label='Woon')
df_mol.plot(x='name', y='n_x', color='Red', kind='scatter',ax=ax, label='xTB', figsize=(20,10))
plt.xticks(rotation=90)


In [ ]:
df_net['diff'] = df_net['Reaction_Energy(kJ/mol)_xtb'] - df_net['Reaction_Energy(kJ/mol)_Woon']
#df_net.plot(x='ID_kida', y='diff', color='Red', kind='scatter',figsize=(20,10))
df_net['diff'].diff().hist(bins=30)

In [ ]:
'''
tmp_w = df_mol['multiplicity_woon'].tolist()
tmp_x = df_mol['multiplicity_xtb'].tolist()

tmp_b = []
for i,item in enumerate(tmp_w):
	if (int(tmp_w[i]) - int(tmp_x[i])) == 0:
		tmp_b.append(True)
	else:
		tmp_b.append(False)

df_mol['bool'] = tmp_b
df_mol_f = df_mol[df_mol['bool'] == False]
df_mol = df_mol.drop(["bool"], axis=1)
'''